In [16]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [17]:
import pickle
import codecs
import collections
import keras.preprocessing
from sklearn.model_selection import KFold
import csv

Using TensorFlow backend.


In [18]:
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,TimeDistributed,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dropout
from keras.models import load_model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding,TimeDistributed,Flatten
from keras.layers import BatchNormalization,Dropout


In [19]:
import seaborn as sns
sns.set(style="darkgrid")
plt.rcParams['font.family']='TH SarabunPSK' #you can change to other font that works for you
plt.rcParams.update({'font.size': 50})

In [20]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [27]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
from pythainlp.tag.named_entity import ThaiNameTagger
ner = ThaiNameTagger()

In [101]:
thai_abb = pd.read_csv('thai_abb.csv')

In [102]:
thai_abb.head()

,อักษรย่อ,ชื่อเต็ม,แท็กหมวดหมู่
0,อ.,อาจารย์,คำนำหน้า|บุคคล
1,ดร.,ด็อกเตอร์,คำนำหน้า|บุคคล
2,ด.ช.,เด็กชาย,คำนำหน้า|บุคคล
3,ด.ญ.,เด็กหญิง,คำนำหน้า|บุคคล
4,อ.ตร.,อธิบดีกรมตำรวจ,อาชีพ|ตำรวจ


In [103]:
abb_set = set(thai_abb['อักษรย่อ'])

In [23]:
# input_pbs = pd.read_csv('notebook4-extractFeature-thispbs.csv')

In [24]:
input_pbs.drop('Unnamed: 0', axis=1, inplace=True)

In [128]:
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.strip(','))

In [131]:
input_pbs['num_date'] = 0
input_pbs['num_email'] = 0
input_pbs['num_law'] = 0
input_pbs['num_len'] = 0
input_pbs['num_location'] = 0
input_pbs['num_money'] = 0
input_pbs['num_organization'] = 0
input_pbs['num_percent'] = 0
input_pbs['num_person'] = 0
input_pbs['num_phone'] = 0
input_pbs['num_time'] = 0
input_pbs['num_url'] = 0

In [209]:
# ner.get_ner('เลื่อน')

[('เลื่อน', 'VERB', 'O')]

In [165]:
# thai_abb[thai_abb['อักษรย่อ']=='คสช.']

,อักษรย่อ,ชื่อเต็ม,แท็กหมวดหมู่
57,คสช.,คณะรักษาความสงบแห่งชาติ,หน่วยงาน|การปกครอง


In [132]:
for index, row in input_pbs.iterrows():
    token = row['token'].split(',')
    for t in token:
        if t in abb_set:
            t = list(thai_abb[thai_abb['อักษรย่อ']==t]['ชื่อเต็ม'])[0]
        elif t.split('.')[0] in abb_set:
            t = list(thai_abb[thai_abb['อักษรย่อ']==t.split('.')[0]]['ชื่อเต็ม'])[0]
        NER = ner.get_ner(t)
        if len(NER) > 1:
            word,pos,n = NER[1]
        else:
            word,pos,n = NER[0]  
        if n == 'I-DATE':
            input_pbs.at[index,'num_date'] += 1
        elif n == 'I-EMAIL':
            input_pbs.at[index,'num_email'] += 1
        elif n == 'I-LAW':
            input_pbs.at[index,'num_law'] += 1
        elif n == 'I-LEN':
            input_pbs.at[index,'num_len'] += 1
        elif n == 'I-LOCATION':
            input_pbs.at[index,'num_location'] += 1
        elif n == 'I-MONEY':
            input_pbs.at[index,'num_money'] += 1
        elif n == 'I-ORGANIZATION':
            input_pbs.at[index,'num_organization'] += 1
        elif n == 'I-PERCENT':
            input_pbs.at[index,'num_percent'] += 1
        elif n == 'I-PERSON':
            input_pbs.at[index,'num_person'] += 1
        elif n == 'I-PHONE':
            input_pbs.at[index,'num_phone'] += 1
        elif n == 'I-TIME':
            input_pbs.at[index,'num_time'] += 1
        elif n == 'I-URL':
            input_pbs.at[index,'num_url'] += 1         

In [182]:
thai_name = pd.read_csv('ThaiName.txt',sep=" ",header=None)
thai_name.columns = ["firstName", "lastName"]
thai_name.dropna(inplace=True)
thai_name_f = pd.read_csv('ThaiName-female.txt',sep=" ",header=None)
thai_name_f.columns = ["firstName"]
thai_name_f.dropna(inplace=True)
thai_name_m = pd.read_csv('ThaiName-man.txt',sep=" ",header=None)
thai_name_m.columns = ["firstName"]
thai_name_m.dropna(inplace=True)

In [208]:
thai_name_m[thai_name_m['firstName']=='เลื่อน']

,firstName
517,เลื่อน


In [155]:
thai_name.head()

,firstName,lastName
0,เกรียงศักดิ์,ละไม
1,กนกวรรณ,ตัณฑเวส
2,เกริกเกียรติ,งานสัมพันธฤทธิ์
3,กฤช,เทพบำรุง
4,กัญญาสรร,ตู้สกุล


In [181]:
thai_name_f.head()

,firstName
0,ศุภรัตน์
1,ชมพูนุช
2,นันท์นริน
3,สุพาพร
4,อุไรวรรณ


In [195]:
thaiNameSet = set(thai_name_f['firstName']).union(set(thai_name_m['firstName']))

In [196]:
len(thaiNameSet)

1378

In [197]:
# 'แก้' in thaiNameSet

False

In [214]:
input_pbs['num_person_2'] = 0

In [216]:
reserve_name = ['สมัคร','ชวน','พายุ','เลื่อน','อำนาจ','สัญญา','นิยม']

In [217]:
for index, row in input_pbs.iterrows():
    token = row['token'].split(',')
    for t in token : 
        if t in thaiNameSet and t not in reserve_name:
            input_pbs.at[index,'num_person_2'] += 1   

In [228]:
pos_df1 = pd.read_csv('sentiment_data/positive_new.txt', header=None)
pos_df1.columns = ["pos_word"]
pos_df1.dropna(inplace=True)
pos_df2 = pd.read_csv('sentiment_data/positive__verbs.txt', header=None)
pos_df2.columns = ["pos_word"]
pos_df2.dropna(inplace=True)
pos_df3 = pd.read_csv('sentiment_data/positive__adjectives.txt', header=None)
pos_df3.columns = ["pos_word"]
pos_df3.dropna(inplace=True)

In [235]:
neg_df1 = pd.read_csv('sentiment_data/negative_new.txt', header=None)
neg_df1.columns = ["neg_word"]
neg_df1.dropna(inplace=True)
neg_df2 = pd.read_csv('sentiment_data/negative_verbs.txt', header=None)
neg_df2.columns = ["neg_word"]
neg_df2.dropna(inplace=True)
neg_df3 = pd.read_csv('sentiment_data/negative_adjectives.txt', header=None)
neg_df3.columns = ["neg_word"]
neg_df3.dropna(inplace=True)

In [230]:
pos_df2.head()

,pos_word
0,ดี
1,ดีใจ
2,กรวด
3,กรวดน้ำ
4,รัก


In [236]:
neg_df1.head()

,neg_word
0,หลบหนี
1,ความขมขื่น
2,ความขมุกขมัว
3,ความขุ่นเคือง
4,ความข้องใจ


In [237]:
pos_word_set = set(pos_df1['pos_word']).union(set(pos_df2['pos_word'])).union(set(pos_df3['pos_word']))
neg_word_set = set(neg_df1['neg_word']).union(set(neg_df2['neg_word'])).union(set(neg_df3['neg_word']))

In [238]:
input_pbs['num_pos'] = 0
input_pbs['num_neg'] = 0

In [239]:
for index, row in input_pbs.iterrows():
    token = row['token'].split(',')
    for t in token : 
        if t in pos_word_set:
            input_pbs.at[index,'num_pos'] += 1  
        elif t in neg_word_set:
            input_pbs.at[index,'num_neg'] += 1  

In [240]:
input_pbs[200:300]

,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar,year-month,month,year,sentiment,timeDelta,num_noun,num_pronoun,num_verb,num_preverb,num_determiner,num_adverb,num_classifier,num_conjunction,num_preposition,num_interjunction,num_prefix,num_ending,num_negator,num_punctuation,is_environment,is_politic,is_health,is_crime,is_social,is_inter,is_sport,is_economic,is_region,is_science,is_art,is_lifestyle,is_disaster,is_south,is_special_report,is_article,is_agriculture,is_education,is_women,is_asian,nnoun_per_ntoken,nverb_per_ntoken,nadverb_per_ntoken,num_date,num_email,num_law,num_len,num_location,num_money,num_organization,num_percent,num_person,num_phone,num_time,num_url,num_person_2,num_pos,num_neg
200,เลือกตั้ง2562 : นายกฯ ส่งสารจัดตั้งรัฐบาลไม่ใช่การต่อสู้กัน แต่เพื่อขจัดคนไม่ดีออกไป,2019-03-29,FRI,13:17,9357,การเมือง,"พล.อ.ประยุทธ์ จันทร์โอชา,เลือกตั้ง2562,สารจากนายกฯ,ThaiPBSnews",13,4,"เลือกตั้ง,2562,นายก,สาร,รัฐบาล,ต่อสู้,ขจัด,คน,ดี",9,40,2562-03,3,2562,neg,36,6,0,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.666667,0.222222,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
201,เลือกตั้ง2562 : เปิดขั้นตอนถอด กกต. ไม่ทันรับรอง ส.ส. 9 พ.ค.นี้,2019-03-29,FRI,13:11,1151,การเมือง,"เลือกตั้ง62,เลือกตั้ง2562,กกต.,เลือกตั้ง 62,ถอดถอน,ThaiPBSnews,ไทยพีบีเอส,ข่าวไทยพีบีเอส,ThaiPBS",13,9,"เลือกตั้ง,2562,ขั้นตอน,ถอด,กกต.,ส.ส.,พ.ค.,นี้",8,38,2562-03,3,2562,neg,36,6,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.750000,0.125000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
202,เลือกตั้ง 2562: เปิดเหตุผล นศ.ร่วมตั้งโต๊ะถอดถอนกกต.,2019-03-29,FRI,13:04,731,การเมือง,"เลือกตั้ง62,จุฬาลงกรณ์มหาวิทยาลัย,กกต.,ถอดถอน,ไทยพีบีเอส,ThaiPBSnews",13,6,"เลือกตั้ง,2562,นศ.,โต๊ะ,ถอดถอน,กกต.",6,30,2562-03,3,2562,neg,36,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
203,"เลือกตั้ง 2562 : ""อลงกรณ์"" ย้ำเลือกแม่ทัพประชาธิปัตย์ต้องหยั่งเสียง",2019-03-29,FRI,12:59,701,การเมือง,"ประชาธิปัตย์,หัวหน้าพรรค,หยั่งเสียง,ฝ่านค้านสร้างสรรค์,ไทยพีบีเอส,ข่าวไทยพีบีเอส,เลือกตั้ง62",12,7,"เลือกตั้ง,2562,อลงกรณ์,ย้ำ,เลือก,แม่ทัพ,ประชาธิปัตย์,หยั่งเสียง",8,56,2562-03,3,2562,pos,36,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.875000,0.125000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204,เตือนเลี่ยงกินหมูดิบ เสี่ยงไข้หูดับ,2019-03-29,FRI,12:36,764,สาธารณสุข,"ไข้หูดับ,หมูดิบ,กรมควบคุมโรค,หูหนวกถาวร,ThaiPBSnews,ThaiPBS,ไทยพีบีเอส,ข่าวไทยพีบีเอส",12,8,"เตือน,เลี่ยง,กิน,หมู,ดิบ,เสี่ยง,ไข้,หูดับ",8,34,2562-03,3,2562,neg,36,5,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.625000,0.375000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
205,เลือกตั้ง 2562 : ผู้สมัคร ส.ส.เพื่อไทย-อนาคตใหม่ ร้อง กกต.อ้างพบทุจริต,2019-03-29,FRI,12:22,1944,การเมือง,"กกต.,อนาคตใหม่,เลือกตั้งใหม่,ไทยพีบีเอส,ข่าวไทยพีบีเอส,เลือกตั้ง62",12,6,"เลือกตั้ง,2562,สมัคร,ส.ส.,ไทย,อนาคต,ร้อง,กกต.,อ้าง,ทุจริต",10,48,2562-03,3,2562,pos,36,7,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.700000,0.300000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
206,เลือกตั้ง2562: ปชป.งัดหลักฐานพบทุจริตซื้อเสียงภาคใต้,2019-03-29,FRI,12:48,9338,การเมือง,"เลือกตั้ง62,ซื้อเสียง,กกต.,พรรคประชาธิปัตย์,เลือกตั้ง2562,ThaiPBSnews",12,6,"เลือกตั้ง,2562,ปชป.,งัด,หลักฐาน,ทุจริต,ซื้อ,เสียง",8,42,2562-03,3,2562,neg,36,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.875000,0.125000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
207,ไม่หาย ! ใช้แอปพลิเคชันสกัดแอบอ้างขึ้นรางวัลล็อตเตอรี่,2019-03-29,FRI,11:53,252,เศรษฐกิจ,"สำนักงานสลากกินแบ่งรัฐบาล,แอปพลิเคชัน,GLO Lottery,ไทยพีบีเอส,ThaiPBSnews",11,5,"หาย,แอปพลิเคชัน,สกัด,แอบ,อ้าง,รางวัล,ล็อตเตอรี่",7,41,2562-03,3,2562,neg,36,4,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.571429,0.428571,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
208,"เลือกตั้ง 2562 : ยื่นยุบพรรค ""อนาคตใหม่- เพื่อชาติ""",2019-03-29,FRI,10:21,25912,การเมือง,"เลือกตั้ง2562,ยุบพรรค,เพื่อไทย,เพื่อชาติ,จตุพร,ธนาธร",10,6,"เลือกตั้ง,2562

In [ ]:
- ner /
- num tag similar to best and worst topic /
- num person from name list /
- num pos/neg word / 
- replace abbreviation /

In [241]:
input_pbs.to_csv('notebook5-extractFeature-thispbs.csv')